In [ ]:
import os
import pandas as pd

from openpyxl import load_workbook

# Set folder path where your Excel files are stored
folder_path = r'path_to_your_folder'  # <-- Replace this with your actual folder path

# To store flagged files and sheet names
issues = []

# Loop through each Excel file in the folder
for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file)
        try:
            wb = load_workbook(file_path, data_only=True)
            sheet_names = wb.sheetnames

            # Check only sheet 2 and 3 (index 1 and 2)
            for sheet_index in [1, 2]:
                if sheet_index < len(sheet_names):
                    ws = wb[sheet_names[sheet_index]]

                    for row in ws.iter_rows(min_row=1, values_only=True):
                        if row[2] == 'Check':  # Column C is index 2 (0-based)
                            # From Column E (index 4) onwards
                            values_to_check = row[4:]
                            if any(val != 0 and val is not None for val in values_to_check):
                                issues.append((file, sheet_names[sheet_index]))
                            break  # No need to search further in this sheet
        except Exception as e:
            print(f"Error processing {file}: {e}")

# Print the issues
if issues:
    print("❌ Files with failed validation:")
    for file_name, sheet_name in issues:
        print(f"File: {file_name}, Sheet: {sheet_name}")
else:
    print("✅ All files passed the validation checks.")


In [4]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox

# Required sheet and column mapping
required_sheets = {
    'RTN': 'RTN',
    'Location': 'Country',
    'GCB': 'GCB'
}

def select_workbook():
    """Prompts the user to select an Excel file."""
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    file_path = filedialog.askopenfilename(
        title="Select Excel Workbook",
        filetypes=[("Excel files", "*.xlsx")]
    )
    if not file_path:
        raise Exception("No file selected.")
    return file_path

def validate_workbook(file_path):
    """Validates presence of required sheets and columns."""
    try:
        xl = pd.ExcelFile(file_path)
    except Exception as e:
        raise Exception(f"Failed to open Excel file: {e}")

    for sheet, required_col in required_sheets.items():
        if sheet not in xl.sheet_names:
            raise Exception(f"Missing required sheet: '{sheet}'")

        df = xl.parse(sheet)
        df.columns = df.columns.str.strip()  # Remove extra spaces

        if required_col not in df.columns:
            raise Exception(f"Sheet '{sheet}' must contain column '{required_col}'")
        
        if df[required_col].isnull().any():
            raise Exception(f"Column '{required_col}' in sheet '{sheet}' contains blank values")

    print("Workbook validation successful.")

if __name__ == "__main__":
    try:
        file_path = select_workbook()
        validate_workbook(file_path)
        print(f"Processing file: {file_path}")
        # Continue with your processing logic here...
    except Exception as e:
        print(f"Error: {e}")
